In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine,MetaData,Table

In [2]:
url = "https://money.cnn.com/data/hotstocks/"
#headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 

In [3]:
res = requests.get(url)

In [4]:
soup = BeautifulSoup(res.content,"html.parser")

In [5]:
soup.title.text

'Hot Stocks - Stock Market Movers - CNNMoney'

In [6]:
b=soup.select( ".wsod_symbol" )

In [24]:
a=soup.find("table",class_= "wsod_dataTable wsod_dataTableBigAlt" )


In [25]:
c=a.select( ".wsod_symbol" )

In [26]:
symbols=[]
for items in c:
    symbols.append(items.text)

In [27]:
asymbol=symbols

In [28]:
asymbol

['CCL', 'BAC', 'XOM', 'F', 'T', 'OXY', 'NCLH', 'DAL', 'VZ', 'RTX']

In [29]:
result =[]

In [30]:
for items in asymbol:
    url="https://finance.yahoo.com/quote/{}".format(items)
    req=requests.get(url.format('1'))
    ysoup = BeautifulSoup(req.text,"html.parser")
    
    #print(items)
    
    name = ysoup.find('h1',class_='D(ib) Fz(18px)')
    na=name.text
    print(na)
    
    tab1=ysoup.find('table',{'class':"W(100%)"})
    se=tab1.find('td',{'data-test': 'OPEN-value'})
    op=se.text
    print("Open:-",op)
    
    #tab1=ysoup.find('table',{'class':"W(100%)"})
    volume=tab1.find('td',{'data-test': 'TD_VOLUME-value'})
    vol=volume.text
    print("volume:-",vol)
    
    tab2=ysoup.find('table',{'class':"W(100%) M(0) Bdcl(c)"})
    per=tab1.find('td',{'data-test': 'PE_RATIO-value'})
    pe=se.text
    print("PERatio:-",pe)
    print("------------")
    
    result.append([na,op,vol,pe])

     

Carnival Corporation & plc (CCL)
Open:- 13.07
volume:- 16,559,190
PERatio:- 13.07
------------
Bank of America Corporation (BAC)
Open:- 25.76
volume:- 9,816,658
PERatio:- 25.76
------------
Exxon Mobil Corporation (XOM)
Open:- 110.44
volume:- 9,153,211
PERatio:- 110.44
------------
Ford Motor Company (F)
Open:- 11.80
volume:- 9,120,566
PERatio:- 11.80
------------
AT&T Inc. (T)
Open:- 14.47
volume:- 8,658,408
PERatio:- 14.47
------------
Occidental Petroleum Corporation (OXY)
Open:- 63.56
volume:- 7,681,419
PERatio:- 63.56
------------
Norwegian Cruise Line Holdings Ltd. (NCLH)
Open:- 15.85
volume:- 6,921,017
PERatio:- 15.85
------------
Delta Air Lines, Inc. (DAL)
Open:- 35.71
volume:- 6,054,495
PERatio:- 35.71
------------
Verizon Communications Inc. (VZ)
Open:- 30.74
volume:- 5,556,446
PERatio:- 30.74
------------
RTX Corporation (RTX)
Open:- 73.20
volume:- 5,603,488
PERatio:- 73.20
------------


In [32]:
print(result)

[['Carnival Corporation & plc (CCL)', '13.07', '16,559,190', '13.07'], ['Bank of America Corporation (BAC)', '25.76', '9,816,658', '25.76'], ['Exxon Mobil Corporation (XOM)', '110.44', '9,153,211', '110.44'], ['Ford Motor Company (F)', '11.80', '9,120,566', '11.80'], ['AT&T Inc. (T)', '14.47', '8,658,408', '14.47'], ['Occidental Petroleum Corporation (OXY)', '63.56', '7,681,419', '63.56'], ['Norwegian Cruise Line Holdings Ltd. (NCLH)', '15.85', '6,921,017', '15.85'], ['Delta Air Lines, Inc. (DAL)', '35.71', '6,054,495', '35.71'], ['Verizon Communications Inc. (VZ)', '30.74', '5,556,446', '30.74'], ['RTX Corporation (RTX)', '73.20', '5,603,488', '73.20']]


In [33]:
colname=['Company','Open_value','Volume','PE_Ratio']
df = pd.DataFrame(data=result,columns=colname)

In [34]:
df

,Company,Open_value,Volume,PE_Ratio
0,Carnival Corporation & plc (CCL),13.07,"16,559,190",13.07
1,Bank of America Corporation (BAC),25.76,"9,816,658",25.76
2,Exxon Mobil Corporation (XOM),110.44,"9,153,211",110.44
3,Ford Motor Company (F),11.80,"9,120,566",11.80
4,AT&T Inc. (T),14.47,"8,658,408",14.47
5,Occidental Petroleum Corporation (OXY),63.56,"7,681,419",63.56
6,Norwegian Cruise Line Holdings Ltd. (NCLH),15.85,"6,921,017",15.85
7,"Delta Air Lines, Inc. (DAL)",35.71,"6,054,495",35.71
8,Verizon Communications Inc. (VZ),30.74,"5,556,446",30.74
9,RTX Corporation (RTX),73.20,"5,603,488",73.20


In [35]:
df.to_csv('project1_2.csv',mode='w')   #exported to csv 

In [36]:
df['Open_value'] = df['Open_value'].astype(str).astype(float)
#df['Volume'] = df['Volume'].replace(',','').astype(int)
df['PE_Ratio'] = df['PE_Ratio'].astype(str).astype(float)


In [37]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Company     10 non-null     object 
 1   Open_value  10 non-null     float64
 2   Volume      10 non-null     object 
 3   PE_Ratio    10 non-null     float64
dtypes: float64(2), object(2)
memory usage: 452.0+ bytes


In [38]:
dbase1 = create_engine('sqlite:///dbase1.db')

In [39]:
df.to_sql(name='stocks',con=dbase1,if_exists='replace', index=False)

10

In [40]:
pd.read_sql(sql='stocks',con=dbase1)

,Company,Open_value,Volume,PE_Ratio
0,Carnival Corporation & plc (CCL),13.07,"16,559,190",13.07
1,Bank of America Corporation (BAC),25.76,"9,816,658",25.76
2,Exxon Mobil Corporation (XOM),110.44,"9,153,211",110.44
3,Ford Motor Company (F),11.80,"9,120,566",11.80
4,AT&T Inc. (T),14.47,"8,658,408",14.47
5,Occidental Petroleum Corporation (OXY),63.56,"7,681,419",63.56
6,Norwegian Cruise Line Holdings Ltd. (NCLH),15.85,"6,921,017",15.85
7,"Delta Air Lines, Inc. (DAL)",35.71,"6,054,495",35.71
8,Verizon Communications Inc. (VZ),30.74,"5,556,446",30.74
9,RTX Corporation (RTX),73.20,"5,603,488",73.20


In [41]:
sqlalchemy.__version__

'1.4.39'

# get a column "company"

In [42]:
table_name = 'stocks'  
column_name = 'Company'

In [43]:
print(f"The column '{column_name}' exists in the table '{table_name}'.")

The column 'Company' exists in the table 'stocks'.


# 

In [44]:
metadata = MetaData()

In [45]:
table = Table(table_name, metadata, autoload=True, autoload_with=dbase1)

In [46]:
connection = dbase1.connect()

In [47]:
select_stmt = table.select().with_only_columns([table.c[column_name]])
result = connection.execute(select_stmt)

In [48]:
for row in result:
    print(row[column_name])

# Close the database connection
connection.close()

Carnival Corporation & plc (CCL)
Bank of America Corporation (BAC)
Exxon Mobil Corporation (XOM)
Ford Motor Company (F)
AT&T Inc. (T)
Occidental Petroleum Corporation (OXY)
Norwegian Cruise Line Holdings Ltd. (NCLH)
Delta Air Lines, Inc. (DAL)
Verizon Communications Inc. (VZ)
RTX Corporation (RTX)


# check an element

In [49]:
table = Table('stocks', metadata, autoload=True, autoload_with=dbase1)
element_to_find = 'PG&E Corporation (PCG)'

# Create a query to fetch a row based on a condition
query = select([table]).where(table.c.Company == element_to_find)

# Execute the query
result = dbase1.execute(query)

# Fetch the row
row = result.fetchone()

if row:
    print("Row found:")
    print(row)
else:
    print("Row not found")

# Close the engine when done
#engine.dispose()

NameError: name 'select' is not defined